In [1]:
from sklearn import cluster, decomposition, preprocessing, feature_selection
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [2]:
centers = pd.read_csv('../../data/interim/Modelling/clusterProfilesTSVD.csv').iloc[:,1:]
centers

,codeSqaleDebtRatio,codeCommentLinesDensity,codeBlockerViolations,codeBugs,resolutionTime,codeFunctionComplexity,codeInfoViolations,codeMajorViolations,sonarCloseTime,timeInProject,...,sonarBug,sonarCritical,sonarInfo,linesRemoved,sonarVulnerability,commitChangeRename,sonarBlocker,commitChangeDelete,jiraDependencyUpgrade,commitChangeUnknown
0,0.00,0.01,0.02,0.26,36271.81,-0.00,-0.42,-5.26,21.89,4.594220e+03,...,0.38,2.59,1.14,-1.28,1.27,-0.03,0.53,-0.04,-0.0,-0.02
1,0.01,-0.01,0.08,-0.19,2735.62,0.01,1.44,12.10,881.42,6.829819e+06,...,0.16,-0.37,-1.19,4.35,-0.79,0.13,-0.81,0.15,-0.0,0.07
2,0.56,0.12,4.88,17.44,-11.56,0.32,23.71,492.61,44421.04,1.039248e+08,...,37.06,216.08,60.02,17.00,92.37,0.50,27.08,0.45,0.0,0.02
3,0.00,0.01,0.01,0.07,1332.37,-0.00,-0.73,-2.93,2326.58,8.100296e+06,...,1.42,11.61,3.76,2.45,5.28,0.00,1.97,-0.05,0.0,-0.05
4,4.57,-0.19,52.62,251.71,34.34,2.71,257.16,6136.34,27218.90,1.955349e+08,...,630.61,4413.41,1416.34,12.75,1904.19,0.87,703.24,0.66,0.0,-0.01


In [3]:
dev = pd.read_csv('../../data/processed/DEVELOPERS_DATA.csv').iloc[:,1:]
cols = ['committer'] + list(centers.columns)
dev = dev.reindex(columns=cols)
dev.head()

,committer,codeSqaleDebtRatio,codeCommentLinesDensity,codeBlockerViolations,codeBugs,resolutionTime,codeFunctionComplexity,codeInfoViolations,codeMajorViolations,sonarCloseTime,...,sonarBug,sonarCritical,sonarInfo,linesRemoved,sonarVulnerability,commitChangeRename,sonarBlocker,commitChangeDelete,jiraDependencyUpgrade,commitChangeUnknown
0,-l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22617.766389,...,0.0,0.0,0.0,14.141744,0.0,0.407407,0.0,1.111111,0.0,0.0
1,1028332163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,4.105769,0.0,0.000000,0.0,0.000000,0.0,0.0
2,A. J. David Bosschaert,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,6.916947,0.0,0.393023,0.0,0.111628,0.0,0.0
3,A195882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,809.977238,...,0.0,0.0,7.0,26.500000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,A744013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,821.388563,...,14.0,25.0,0.0,3.700000,3.0,1.000000,0.0,0.000000,0.0,0.0


In [4]:
dev2 = dev.iloc[:,1:]
print(dev2.shape)
dev2.head()

(2460, 94)


,codeSqaleDebtRatio,codeCommentLinesDensity,codeBlockerViolations,codeBugs,resolutionTime,codeFunctionComplexity,codeInfoViolations,codeMajorViolations,sonarCloseTime,timeInProject,...,sonarBug,sonarCritical,sonarInfo,linesRemoved,sonarVulnerability,commitChangeRename,sonarBlocker,commitChangeDelete,jiraDependencyUpgrade,commitChangeUnknown
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22617.766389,4235880.0,...,0.0,0.0,0.0,14.141744,0.0,0.407407,0.0,1.111111,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,77939.0,...,0.0,0.0,0.0,4.105769,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,173937105.0,...,0.0,0.0,0.0,6.916947,0.0,0.393023,0.0,0.111628,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,809.977238,0.0,...,0.0,0.0,7.0,26.500000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,821.388563,351970.0,...,14.0,25.0,0.0,3.700000,3.0,1.000000,0.0,0.000000,0.0,0.0


Assign to each developer a label of its cluster

In [5]:
kmeans = cluster.KMeans(n_clusters=5, init=centers, n_init=1, max_iter=1).fit(dev2)
kmeans.cluster_centers = np.array(centers)

In [6]:
clusters = kmeans.predict(dev2)

In [7]:
dev['cluster'] = clusters
dev.head()

,committer,codeSqaleDebtRatio,codeCommentLinesDensity,codeBlockerViolations,codeBugs,resolutionTime,codeFunctionComplexity,codeInfoViolations,codeMajorViolations,sonarCloseTime,...,sonarCritical,sonarInfo,linesRemoved,sonarVulnerability,commitChangeRename,sonarBlocker,commitChangeDelete,jiraDependencyUpgrade,commitChangeUnknown,cluster
0,-l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22617.766389,...,0.0,0.0,14.141744,0.0,0.407407,0.0,1.111111,0.0,0.0,1
1,1028332163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,4.105769,0.0,0.000000,0.0,0.000000,0.0,0.0,0
2,A. J. David Bosschaert,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,6.916947,0.0,0.393023,0.0,0.111628,0.0,0.0,4
3,A195882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,809.977238,...,0.0,7.0,26.500000,0.0,0.000000,0.0,0.000000,0.0,0.0,0
4,A744013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,821.388563,...,25.0,0.0,3.700000,3.0,1.000000,0.0,0.000000,0.0,0.0,0


Within cluster variance

In [8]:
def WCV(dev, centers):
  WCV = np.zeros(5)
  for i in range(5):  # clusters
    X = dev[dev.cluster==i].iloc[:,1:-1]
    c = [np.array(centers)[i]]
    d = distance.cdist(X, c)
    WCV[i] = d.sum()/d.shape[0]
  return [WCV, WCV.sum()]

In [9]:
cluster, total = WCV(dev, centers)
print(cluster)
print(total)

[  107036.88884768  3897443.04091219 23130150.52049431 27642915.56625457
 64784179.54550792]
119561725.56201667


Between cluster variance

In [10]:
def BCV(dev, centers):
  BCV = np.zeros(5)
  x = [np.array(dev.iloc[:,1:-1].mean())]
  for i in range(5):
    n = dev[dev.cluster==i].shape[0]
    c = [np.array(centers)[i]]
    d = distance.cdist(c, x)
    BCV[i] = n*d.sum()
  return [BCV, BCV.sum()]

In [11]:
cluster, total = BCV(dev, centers)
print(cluster)
print(total)

[3.02542518e+10 1.60925046e+09 1.45831849e+10 2.17787469e+09
 1.00199655e+10]
58644527338.20222


# **Davies–Bouldin index:**

In [12]:
def DB(dev, centers):
  wcv, _ = WCV(dev, centers) # mean distance of all elements in cluster to centroid
  DBC = np.zeros((5,5)) # distance between centroids
  DavisBouldin = 0
  for i in range(5):  # clusters
    max = 0
    for j in range(5):
      ci = [np.array(centers)[i]]
      cj = [np.array(centers)[j]]
      d = distance.cdist(ci, cj)
      DBC[i,j] = d.sum()

      if i != j:
        val = (wcv[i]+wcv[j])/DBC[i,j]
        if val > max:
          max = val
    DavisBouldin += max
  return DavisBouldin/5

In [13]:
DavisBouldinIndex = DB(dev, centers)
DavisBouldinIndex

10.96740426346652

## **Types of issues**:

In [14]:
centers[["codeBlockerViolations", "codeInfoViolations",	"codeMajorViolations", "codeBugs", "codeViolations", "codeVulnerabilities",	"codeCodeSmells", "codeCriticalViolations",	"codeMinorViolations", "inducedSZZIssues",	"inducedSonarIssues", ]]

,codeBlockerViolations,codeInfoViolations,codeMajorViolations,codeBugs,codeViolations,codeVulnerabilities,codeCodeSmells,codeCriticalViolations,codeMinorViolations,inducedSZZIssues,inducedSonarIssues
0,0.02,-0.42,-5.26,0.26,-22.15,-0.50,-21.84,-0.84,-15.32,0.01,0.39
1,0.08,1.44,12.10,-0.19,43.10,0.55,42.57,1.38,27.34,0.03,0.42
2,4.88,23.71,492.61,17.44,1311.99,28.92,1265.35,72.81,716.68,1.46,47.84
3,0.01,-0.73,-2.93,0.07,-13.98,0.06,-14.03,0.33,-10.27,0.07,3.26
4,52.62,257.16,6136.34,251.71,16295.13,410.71,15632.01,810.45,9035.32,17.27,419.84


## **3 clusters:**

In [15]:
from sklearn import cluster, decomposition, preprocessing, feature_selection
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [16]:
centers3 = pd.read_csv('../../data/interim/Modelling/3clusterProfilesTSVD.csv').iloc[:,1:]
centers3

,codeSqaleDebtRatio,codeCommentLinesDensity,codeBlockerViolations,codeBugs,codeFunctionComplexity,sonarCloseTime,timeInProject,resolutionTime,codeInfoViolations,codeMajorViolations,...,sonarBlocker,commitChangeRename,linesRemoved,jiraDocumentation,jiraTechnicalTask,jiraStory,commitChangeDelete,jiraQuestion,jiraDependencyUpgrade,commitChangeUnknown
0,0.57,0.11,6.17,20.06,0.34,43388.31,1.038724e+08,-3.98,25.07,561.07,...,63.76,0.49,16.84,-0.00,-0.01,-0.01,0.44,-0.0,-0.0,0.02
1,4.68,-0.20,30.93,221.32,2.45,30980.04,1.970348e+08,-2.42,253.63,5353.44,...,49.82,1.02,11.94,0.00,0.01,0.00,0.72,-0.0,0.0,-0.02
2,0.01,-0.01,0.07,-0.12,0.01,757.55,5.926980e+06,6135.12,1.22,10.29,...,-0.66,0.12,3.73,0.01,0.06,0.17,0.12,0.0,-0.0,0.06


In [17]:
centers3["refactoringExtractMethod"]

0      6.40
1    157.14
2      0.25
Name: refactoringExtractMethod, dtype: float64